In [1]:
'''Load Libs'''
import numpy as np
import pandas as pd
import os.path
from pathlib import Path
from collections import Counter
import random
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
from time import time

In [2]:
'''Load Data And Identify Classes'''
PathTrain =  os.getcwd() + "/data/train.csv"
PathTest =  os.getcwd() + "/data/test.csv"
DFTrain = pd.read_csv(PathTrain, header=None)
DFTest = pd.read_csv(PathTest, header=None)
totalClasses = len(Counter(DFTrain.iloc[:,-1]))

In [3]:
DTrain = DFTrain.iloc[:,:-1]
DTrain = np.c_[np.ones(len(DTrain)), DTrain]
DTest = DFTest.iloc[:,:-1]
DTest = np.c_[np.ones(len(DTest)), DTest]
LabelsTest = np.array(DFTest.iloc[:,-1])

In [4]:
'''One Hot Encoding for Train and Test Labels'''
oneHotTrainLables = np.zeros((len(DFTrain), totalClasses))
oneHotTestLables = np.zeros((len(DFTest), totalClasses))

for i in range(len(DFTrain)):
    oneHotTrainLables[i][DFTrain.iloc[i, -1]] = 1
    
for i in range(len(DFTest)):
    oneHotTestLables[i][DFTest.iloc[i, -1]] = 1

In [45]:
def sigmoid(z):
    #Sigmoid Function
    return 1/(1+np.exp(-z))

def ReLU(z):
    #Rectified Linear Unit
    return np.maximum(0,z)

def delSigmoid(z):
    #oj = sigmoid(z)
    return np.multiply(z, (1-z))

def delReLU(z):
    z = np.matrix(z)
    z[z > 0] = 1
    z[z <= 0] = 0    
    return z
    
    
class Layer:    
    
    def __init__(self, perceptron_units, n_inputs, activation_func, layer_type):
        self.type = layer_type
        self.activation = activation_func
        self.perceptron_units = perceptron_units
        self.inputs = None
        self.output = None
        #self.weights = 0.005*np.random.rand(n_inputs, perceptron_units)  
        self.weights = np.random.normal(0,0.05, perceptron_units * (n_inputs)).reshape(n_inputs, perceptron_units)
        self.delta = None
        
    def __repr__(self):
        representation = (self.type, self.perceptron_units, self.weights.shape, self.activation)
        return "<%s Layer | Num_Perceptrons_Units = %d, Weights = %s, Activation = %s>" % representation
    

class NeuralNetwork:
    
    def __init__(self, list_hidden_layers, op_layers_activation, hidden_layers_activation):
        np.random.seed(525)
        self.total_layers = len(list_hidden_layers) + 1
        self.nodes_hidden_layers = list_hidden_layers
        self.layers = []
        
        for i in range(len(list_hidden_layers)):
            if i == 0:
                layer = Layer(list_hidden_layers[i], n, hidden_layers_activation, "Hidden")
                self.layers.append(layer)
            else:
                layer = Layer(list_hidden_layers[i], list_hidden_layers[i-1], hidden_layers_activation, "Hidden")                
                self.layers.append(layer)
        
        layer = Layer(r, list_hidden_layers[-1], op_layers_activation, "Output")
        self.layers.append(layer)
        
        
    def __repr__(self):
        layers = self.layers
        rep = ""
        print("Neural Network:")
        for i in range(len(layers)):
            rep += "Layer %d: %s\n" % (i, layers[i])
        return rep
    
    def forwardFeed(self, ip_data):
        layer = self.layers[0]
        layer.inputs = np.matrix(ip_data)
        layer.netj = np.matmul(layer.inputs, np.matrix(layer.weights))
        layer.output = self.activation(layer.netj, layer.activation)
        for i in range(1, len(self.layers), 1):
            layer = self.layers[i]
            layer.inputs = self.layers[i-1].output
            layer.netj = np.matmul(layer.inputs, np.matrix(layer.weights))
            layer.output = self.activation(layer.netj, layer.activation)
    
    def backwardPropagation(self, labels):
        output_layer = self.layers[-1]
        error = labels - output_layer.output
        delOj = self.delActivation(output_layer.output, output_layer.activation)
        output_layer.delta = -1*np.multiply(error, delOj)
        for i in reversed(range(self.total_layers-1)):
            current_layer = self.layers[i]
            next_layer = self.layers[i + 1]
            error = np.matmul(next_layer.delta, next_layer.weights.T)
            delOj = self.delActivation(current_layer.output, current_layer.activation)
            current_layer.delta = np.multiply(error, delOj)
            
    def activation(self, x, activation):
        if activation == "Sigmoid":
            return sigmoid(x)
        elif activation == "ReLU":
            return ReLU(x)
    
    def delActivation(self, x, activation):
        if activation == "Sigmoid":            
            return delSigmoid(x)
        
        elif activation == "ReLU":
            return delReLU(x)
        
    def updateParams(self, lr, bSize):
        layers = self.layers
        for layer in layers:
            #print(layer.inputs.shape, layer.delta.shape)
            gradient = np.matmul(layer.inputs.T, layer.delta)
            layer.weights = layer.weights - (lr/bSize)*gradient
    
            
    def meanSquaredError(self, Y, avg=True):
        div = 1
        #if avg:
        div = len(Y)
        
        op_layer_labels = self.layers[-1].output
        error = Y - op_layer_labels
        error = np.square(error)
        error = np.sum(error)/(2*div)
        return error
    
    
    def fit(self, X, Labels, eta = 0.1, batch_size=100, max_epoch = 10000, epsilon=1e-9):
        lr = eta
        data = X
        labels = Labels
        
        epoch = 0
        self.forwardFeed(data)
        error_prev = self.meanSquaredError(labels)
        epoch_error_list = [error_prev]
        t0 = time()
        while epoch < max_epoch:
            t1 = time()
            epoch += 1
            
            data, labels = shuffle(data, labels)
            
            if eta == 0:
                lr = 0.5/np.sqrt(epoch)
                
            for batch_start in range(0, len(data), batch_size):
                batch_end = batch_start + batch_size
                Xb = data[batch_start : batch_end]
                Yb = labels[batch_start : batch_end]
                
                self.forwardFeed(Xb)
                self.backwardPropagation(Yb)
                
                self.updateParams(lr, batch_size)
                #return
                
            t2 = time()   
            self.forwardFeed(data)
            error = self.meanSquaredError(labels)
            deltaError = error - error_prev
            epoch_error_list.append(error)            
            print("$$ Epoch: {} | Error = {} | DeltaError = {} | LR = {} | Epoch Train Time = {}Sec"
                  .format(epoch, round(error,6), round(deltaError,6), lr, round(t2-t1,2)))
            if np.abs(deltaError) < epsilon:
                break
            error_prev = error
        
        t4 = time()
        print("\n%% Total Epochs ={} | Epsilon = {} | Total Learning Time = {}Min"
              .format(epoch, epsilon, round((t4-t0)/60,2)))
        
    def predict(self, data, labels):
        self.forwardFeed(data)
        outputs = self.layers[-1].output
        prediction = []
        for x in outputs:
            prediction.append(np.argmax(x))
        print(len(prediction), labels.shape)
        accu_score = accuracy_score(labels, prediction)
        print("Accuracy Score: ", 100*accu_score)


In [47]:
'''Global Params'''
M = 100 #MiniBatch Size
n = len(DTrain[0])
nodesInHiddenLayers = [50]
r = totalClasses
neu_net = NeuralNetwork(nodesInHiddenLayers, "Sigmoid", "Sigmoid")
print(neu_net)
neu_net.fit(DTrain, oneHotTrainLables, eta=0.1, max_epoch=10000, epsilon=1e-8)
neu_net.predict(DTest, LabelsTest)

Neural Network:
Layer 0: <Hidden Layer | Num_Perceptrons_Units = 50, Weights = (785, 50), Activation = Sigmoid>
Layer 1: <Output Layer | Num_Perceptrons_Units = 26, Weights = (50, 26), Activation = Sigmoid>

$$ Epoch: 1 | Error = 0.481675 | DeltaError = -2.695674 | LR = 0.1 | Epoch Train Time = 0.23Sec
$$ Epoch: 2 | Error = 0.479711 | DeltaError = -0.001965 | LR = 0.1 | Epoch Train Time = 0.16Sec
$$ Epoch: 3 | Error = 0.478959 | DeltaError = -0.000751 | LR = 0.1 | Epoch Train Time = 0.17Sec
$$ Epoch: 4 | Error = 0.478201 | DeltaError = -0.000758 | LR = 0.1 | Epoch Train Time = 0.16Sec
$$ Epoch: 5 | Error = 0.477286 | DeltaError = -0.000915 | LR = 0.1 | Epoch Train Time = 0.15Sec
$$ Epoch: 6 | Error = 0.476128 | DeltaError = -0.001158 | LR = 0.1 | Epoch Train Time = 0.18Sec
$$ Epoch: 7 | Error = 0.474697 | DeltaError = -0.00143 | LR = 0.1 | Epoch Train Time = 0.17Sec
$$ Epoch: 8 | Error = 0.473121 | DeltaError = -0.001576 | LR = 0.1 | Epoch Train Time = 0.18Sec
$$ Epoch: 9 | Error = 0.4

$$ Epoch: 84 | Error = 0.161929 | DeltaError = -0.001801 | LR = 0.1 | Epoch Train Time = 0.19Sec
$$ Epoch: 85 | Error = 0.162682 | DeltaError = 0.000753 | LR = 0.1 | Epoch Train Time = 0.18Sec
$$ Epoch: 86 | Error = 0.161984 | DeltaError = -0.000698 | LR = 0.1 | Epoch Train Time = 0.18Sec
$$ Epoch: 87 | Error = 0.167444 | DeltaError = 0.005461 | LR = 0.1 | Epoch Train Time = 0.19Sec
$$ Epoch: 88 | Error = 0.156259 | DeltaError = -0.011186 | LR = 0.1 | Epoch Train Time = 0.18Sec
$$ Epoch: 89 | Error = 0.159799 | DeltaError = 0.003541 | LR = 0.1 | Epoch Train Time = 0.18Sec
$$ Epoch: 90 | Error = 0.159204 | DeltaError = -0.000595 | LR = 0.1 | Epoch Train Time = 0.19Sec
$$ Epoch: 91 | Error = 0.156958 | DeltaError = -0.002247 | LR = 0.1 | Epoch Train Time = 0.16Sec
$$ Epoch: 92 | Error = 0.158665 | DeltaError = 0.001707 | LR = 0.1 | Epoch Train Time = 0.2Sec
$$ Epoch: 93 | Error = 0.154674 | DeltaError = -0.00399 | LR = 0.1 | Epoch Train Time = 0.18Sec
$$ Epoch: 94 | Error = 0.152427 | De

$$ Epoch: 169 | Error = 0.122103 | DeltaError = 0.003014 | LR = 0.1 | Epoch Train Time = 0.28Sec
$$ Epoch: 170 | Error = 0.118635 | DeltaError = -0.003468 | LR = 0.1 | Epoch Train Time = 0.27Sec
$$ Epoch: 171 | Error = 0.121392 | DeltaError = 0.002757 | LR = 0.1 | Epoch Train Time = 0.28Sec
$$ Epoch: 172 | Error = 0.122366 | DeltaError = 0.000974 | LR = 0.1 | Epoch Train Time = 0.26Sec
$$ Epoch: 173 | Error = 0.119639 | DeltaError = -0.002727 | LR = 0.1 | Epoch Train Time = 0.29Sec
$$ Epoch: 174 | Error = 0.120117 | DeltaError = 0.000478 | LR = 0.1 | Epoch Train Time = 0.29Sec
$$ Epoch: 175 | Error = 0.115379 | DeltaError = -0.004738 | LR = 0.1 | Epoch Train Time = 0.28Sec
$$ Epoch: 176 | Error = 0.117881 | DeltaError = 0.002502 | LR = 0.1 | Epoch Train Time = 0.29Sec
$$ Epoch: 177 | Error = 0.118618 | DeltaError = 0.000737 | LR = 0.1 | Epoch Train Time = 0.28Sec
$$ Epoch: 178 | Error = 0.115709 | DeltaError = -0.00291 | LR = 0.1 | Epoch Train Time = 0.27Sec
$$ Epoch: 179 | Error = 0.1

$$ Epoch: 254 | Error = 0.100709 | DeltaError = 0.002461 | LR = 0.1 | Epoch Train Time = 0.35Sec
$$ Epoch: 255 | Error = 0.102773 | DeltaError = 0.002064 | LR = 0.1 | Epoch Train Time = 0.36Sec
$$ Epoch: 256 | Error = 0.099018 | DeltaError = -0.003755 | LR = 0.1 | Epoch Train Time = 0.35Sec
$$ Epoch: 257 | Error = 0.098136 | DeltaError = -0.000882 | LR = 0.1 | Epoch Train Time = 0.35Sec
$$ Epoch: 258 | Error = 0.098685 | DeltaError = 0.000549 | LR = 0.1 | Epoch Train Time = 0.36Sec
$$ Epoch: 259 | Error = 0.09906 | DeltaError = 0.000375 | LR = 0.1 | Epoch Train Time = 0.32Sec
$$ Epoch: 260 | Error = 0.103053 | DeltaError = 0.003993 | LR = 0.1 | Epoch Train Time = 0.35Sec
$$ Epoch: 261 | Error = 0.104501 | DeltaError = 0.001448 | LR = 0.1 | Epoch Train Time = 0.34Sec
$$ Epoch: 262 | Error = 0.09982 | DeltaError = -0.004681 | LR = 0.1 | Epoch Train Time = 0.36Sec
$$ Epoch: 263 | Error = 0.097828 | DeltaError = -0.001992 | LR = 0.1 | Epoch Train Time = 0.32Sec
$$ Epoch: 264 | Error = 0.10

$$ Epoch: 339 | Error = 0.09105 | DeltaError = 0.002083 | LR = 0.1 | Epoch Train Time = 0.37Sec
$$ Epoch: 340 | Error = 0.086715 | DeltaError = -0.004336 | LR = 0.1 | Epoch Train Time = 0.43Sec
$$ Epoch: 341 | Error = 0.089018 | DeltaError = 0.002303 | LR = 0.1 | Epoch Train Time = 0.39Sec
$$ Epoch: 342 | Error = 0.091999 | DeltaError = 0.002982 | LR = 0.1 | Epoch Train Time = 0.36Sec
$$ Epoch: 343 | Error = 0.093673 | DeltaError = 0.001674 | LR = 0.1 | Epoch Train Time = 0.35Sec
$$ Epoch: 344 | Error = 0.094275 | DeltaError = 0.000601 | LR = 0.1 | Epoch Train Time = 0.4Sec
$$ Epoch: 345 | Error = 0.087687 | DeltaError = -0.006588 | LR = 0.1 | Epoch Train Time = 0.3Sec
$$ Epoch: 346 | Error = 0.08929 | DeltaError = 0.001604 | LR = 0.1 | Epoch Train Time = 0.26Sec
$$ Epoch: 347 | Error = 0.094358 | DeltaError = 0.005068 | LR = 0.1 | Epoch Train Time = 0.29Sec
$$ Epoch: 348 | Error = 0.08976 | DeltaError = -0.004599 | LR = 0.1 | Epoch Train Time = 0.3Sec
$$ Epoch: 349 | Error = 0.090962 

...

$$ Epoch: 2282 | Error = 0.030853 | DeltaError = -3e-06 | LR = 0.1 | Epoch Train Time = 0.63Sec
$$ Epoch: 2283 | Error = 0.030851 | DeltaError = -3e-06 | LR = 0.1 | Epoch Train Time = 0.49Sec
$$ Epoch: 2284 | Error = 0.030848 | DeltaError = -2e-06 | LR = 0.1 | Epoch Train Time = 0.62Sec
$$ Epoch: 2285 | Error = 0.030846 | DeltaError = -2e-06 | LR = 0.1 | Epoch Train Time = 0.6Sec
$$ Epoch: 2286 | Error = 0.030843 | DeltaError = -4e-06 | LR = 0.1 | Epoch Train Time = 0.64Sec
$$ Epoch: 2287 | Error = 0.030841 | DeltaError = -2e-06 | LR = 0.1 | Epoch Train Time = 0.46Sec
$$ Epoch: 2288 | Error = 0.030838 | DeltaError = -2e-06 | LR = 0.1 | Epoch Train Time = 0.4Sec
$$ Epoch: 2289 | Error = 0.030834 | DeltaError = -4e-06 | LR = 0.1 | Epoch Train Time = 0.43Sec
$$ Epoch: 2290 | Error = 0.030832 | DeltaError = -2e-06 | LR = 0.1 | Epoch Train Time = 0.44Sec
$$ Epoch: 2291 | Error = 0.03083 | DeltaError = -3e-06 | LR = 0.1 | Epoch Train Time = 0.54Sec
$$ Epoch: 2292 | Error = 0.030827 | DeltaEr

$$ Epoch: 2368 | Error = 0.030638 | DeltaError = -3e-06 | LR = 0.1 | Epoch Train Time = 0.43Sec
$$ Epoch: 2369 | Error = 0.030635 | DeltaError = -3e-06 | LR = 0.1 | Epoch Train Time = 0.47Sec
$$ Epoch: 2370 | Error = 0.030633 | DeltaError = -2e-06 | LR = 0.1 | Epoch Train Time = 0.48Sec
$$ Epoch: 2371 | Error = 0.03063 | DeltaError = -3e-06 | LR = 0.1 | Epoch Train Time = 0.43Sec
$$ Epoch: 2372 | Error = 0.030628 | DeltaError = -2e-06 | LR = 0.1 | Epoch Train Time = 0.41Sec
$$ Epoch: 2373 | Error = 0.030625 | DeltaError = -3e-06 | LR = 0.1 | Epoch Train Time = 0.44Sec
$$ Epoch: 2374 | Error = 0.030623 | DeltaError = -3e-06 | LR = 0.1 | Epoch Train Time = 0.44Sec
$$ Epoch: 2375 | Error = 0.03062 | DeltaError = -2e-06 | LR = 0.1 | Epoch Train Time = 0.38Sec
$$ Epoch: 2376 | Error = 0.030618 | DeltaError = -2e-06 | LR = 0.1 | Epoch Train Time = 0.36Sec
$$ Epoch: 2377 | Error = 0.030615 | DeltaError = -3e-06 | LR = 0.1 | Epoch Train Time = 0.54Sec
$$ Epoch: 2378 | Error = 0.030612 | DeltaE

$$ Epoch: 2454 | Error = 0.030421 | DeltaError = -2e-06 | LR = 0.1 | Epoch Train Time = 0.46Sec
$$ Epoch: 2455 | Error = 0.030418 | DeltaError = -3e-06 | LR = 0.1 | Epoch Train Time = 0.44Sec
$$ Epoch: 2456 | Error = 0.030416 | DeltaError = -3e-06 | LR = 0.1 | Epoch Train Time = 0.4Sec
$$ Epoch: 2457 | Error = 0.030412 | DeltaError = -3e-06 | LR = 0.1 | Epoch Train Time = 0.38Sec
$$ Epoch: 2458 | Error = 0.03041 | DeltaError = -3e-06 | LR = 0.1 | Epoch Train Time = 0.46Sec
$$ Epoch: 2459 | Error = 0.030408 | DeltaError = -2e-06 | LR = 0.1 | Epoch Train Time = 0.41Sec
$$ Epoch: 2460 | Error = 0.030408 | DeltaError = -0.0 | LR = 0.1 | Epoch Train Time = 0.39Sec
$$ Epoch: 2461 | Error = 0.030404 | DeltaError = -4e-06 | LR = 0.1 | Epoch Train Time = 0.42Sec
$$ Epoch: 2462 | Error = 0.030402 | DeltaError = -2e-06 | LR = 0.1 | Epoch Train Time = 0.46Sec
$$ Epoch: 2463 | Error = 0.0304 | DeltaError = -2e-06 | LR = 0.1 | Epoch Train Time = 0.38Sec
$$ Epoch: 2464 | Error = 0.030397 | DeltaError

$$ Epoch: 2540 | Error = 0.030226 | DeltaError = -1e-06 | LR = 0.1 | Epoch Train Time = 0.35Sec
$$ Epoch: 2541 | Error = 0.030223 | DeltaError = -3e-06 | LR = 0.1 | Epoch Train Time = 0.31Sec
$$ Epoch: 2542 | Error = 0.030221 | DeltaError = -3e-06 | LR = 0.1 | Epoch Train Time = 0.31Sec
$$ Epoch: 2543 | Error = 0.030219 | DeltaError = -1e-06 | LR = 0.1 | Epoch Train Time = 0.27Sec
$$ Epoch: 2544 | Error = 0.030217 | DeltaError = -2e-06 | LR = 0.1 | Epoch Train Time = 0.31Sec
$$ Epoch: 2545 | Error = 0.030215 | DeltaError = -2e-06 | LR = 0.1 | Epoch Train Time = 0.34Sec
$$ Epoch: 2546 | Error = 0.030213 | DeltaError = -1e-06 | LR = 0.1 | Epoch Train Time = 0.3Sec
$$ Epoch: 2547 | Error = 0.030211 | DeltaError = -2e-06 | LR = 0.1 | Epoch Train Time = 0.29Sec
$$ Epoch: 2548 | Error = 0.030208 | DeltaError = -3e-06 | LR = 0.1 | Epoch Train Time = 0.29Sec
$$ Epoch: 2549 | Error = 0.030206 | DeltaError = -2e-06 | LR = 0.1 | Epoch Train Time = 0.27Sec
$$ Epoch: 2550 | Error = 0.030204 | Delta

$$ Epoch: 2626 | Error = 0.030044 | DeltaError = -4e-06 | LR = 0.1 | Epoch Train Time = 0.38Sec
$$ Epoch: 2627 | Error = 0.030043 | DeltaError = -1e-06 | LR = 0.1 | Epoch Train Time = 0.39Sec
$$ Epoch: 2628 | Error = 0.030041 | DeltaError = -2e-06 | LR = 0.1 | Epoch Train Time = 0.46Sec
$$ Epoch: 2629 | Error = 0.030038 | DeltaError = -3e-06 | LR = 0.1 | Epoch Train Time = 0.42Sec
$$ Epoch: 2630 | Error = 0.030036 | DeltaError = -2e-06 | LR = 0.1 | Epoch Train Time = 0.36Sec
$$ Epoch: 2631 | Error = 0.030034 | DeltaError = -2e-06 | LR = 0.1 | Epoch Train Time = 0.47Sec
$$ Epoch: 2632 | Error = 0.030033 | DeltaError = -2e-06 | LR = 0.1 | Epoch Train Time = 0.42Sec
$$ Epoch: 2633 | Error = 0.03003 | DeltaError = -2e-06 | LR = 0.1 | Epoch Train Time = 0.38Sec
$$ Epoch: 2634 | Error = 0.030028 | DeltaError = -2e-06 | LR = 0.1 | Epoch Train Time = 0.38Sec
$$ Epoch: 2635 | Error = 0.030026 | DeltaError = -3e-06 | LR = 0.1 | Epoch Train Time = 0.4Sec
$$ Epoch: 2636 | Error = 0.030025 | DeltaE

In [62]:
def prediction(net, data, lbl):
    net.forwardFeed(data)
    outputs = net.layers[-1].output
    prediction = []
    for x in outputs:
        prediction.append(np.argmax(x))
    print(len(prediction), lbl.shape)
    accu_score = accuracy_score(lbl, prediction)
    print("Accuracy= ",accu_score)
prediction(neu_net, DTest, LabelsTest)

6500 (6500,)
Accuracy=  0.6575384615384615


In [42]:
#for layer in neu_net.layers:
neu_net.forwardFeed(DTrain)
layer = neu_net.layers[-1]
for x in layer.output:
    print(np.amax(x))

0.9574965608808638
0.9598815182687106
0.9715072274530074
0.9947492490063464
0.959231388849291
0.9951440532587261
0.9809080617540177
0.9896246420973588
0.9153667210798168
0.992969753786681
0.9981044205572329
0.9355051887869412
0.9921349059892898
0.8823832200701214
0.9954868734068857
0.9940287842713526
0.9633531984936602
0.9963201427429993
0.9986275947143104
0.9653733933674434
0.9831577060576818
0.9588271222967996
0.9885018466661959
0.9990097842251427
0.9968026686284973
0.15885816189179364
0.9984893360591487
0.9283550233150702
0.9772795114396674
0.18389109737959275
0.2966082890310889
0.9931268589655947
0.751149067838081
0.9779459661374764
0.9964525546707461
0.9962631601447332
0.964698660839957
0.9963507660837544
0.9454058016404548
0.9995183472195573
0.9895941014584699
0.9825490783943021
0.9786388723204049
0.9614511678844883
0.9790501218740826
0.9957116121237091
0.9734377517215334
0.9965624438421532
0.9257140835452006
0.7261369217014743
0.8848640039233276
0.9461636943625348
0.984298402532

0.999549824940158
0.9583165958115486
0.9866799711850157
0.9954757874882166
0.8209919122328607
0.9935633035149655
0.9622080587604429
0.9910711052765916
0.9657536665348124
0.9964553862764024
0.9930448563627859
0.9868115532092695
0.9775910383090698
0.9926318453886384
0.9588293652156331
0.11629515464341009
0.9973019900538026
0.9765974114038265
0.9289610468336756
0.9990300554766935
0.9784420205356531
0.9992865914888858
0.9870485008783597
0.9923298893538249
0.9515923003605694
0.9901749763772063
0.9275205299560039
0.984771314468094
0.9953843260607674
0.9364266426922657
0.15062100783431842
0.970622338008795
0.9983766533262356
0.9992176820538983
0.9937878321442973
0.9829620525574917
0.1135654778456229
0.960144465443138
0.989353530739217
0.8639473999625259
0.984598265674392
0.9342789374776173
0.9417658989267004
0.9866412731706173
0.9807626927990905
0.9422606103428718
0.996246208029141
0.9889280225572937
0.13188133230167118
0.9959688321900405
0.9930194064888934
0.10964032975712434
0.9974128300291

0.9990731454391749
0.9659882618966247
0.9960074275516607
0.941788362156794
0.8806603415516308
0.999433608616907
0.988799226310162
0.997724836239478
0.9907051255512678
0.9839870449749578
0.9588366385584915
0.9552442550777445
0.9982839733544133
0.9806800032018735
0.9189100100263156
0.9955767092835648
0.9786175599454197
0.9905559554797658
0.9926653951295042
0.9980756842379509
0.9974335310452737
0.9912363553624038
0.9664221347891023
0.986556859762788
0.9952408747688375
0.9317324055505014
0.9973078922339078
0.9872200716305767
0.9822929461817455
0.99219579397182
0.9981799922028751
0.9602638252823765
0.6544153487944524
0.9974157581490356
0.9816560264393934
0.9907132410282733
0.9788975153587594
0.08770192157611748
0.9751405426417964
0.9959457597321821
0.9900720570347098
0.9953509652393753
0.989150216773803
0.9813945340048795
0.9867300254350235
0.9779206639544359
0.9921601739304261
0.963887620438727
0.8609685539427605
0.9942537413224118
0.8665775140538479
0.8968082607672817
0.9981886375041656
0

0.9990730949771347
0.9880146117303191
0.993122656293516
0.913783075662218
0.9956693485377958
0.9268771624532435
0.9839268507685366
0.9432632973168988
0.987844288611153
0.9835620944720079
0.9378926014857123
0.9899260613644782
0.9860620751987377
0.9979369889013134
0.9798408597390246
0.9971340913261371
0.9774468953257035
0.9786680200654018
0.9982189957748637
0.991927029117033
0.9976569913295753
0.9990289910413533
0.9991257133261955
0.9714317418470496
0.9081945754896817
0.9922634270983182
0.9961309183425257
0.9835067857733272
0.9985426801314583
0.9926570391636708
0.9953906774275059
0.9746651351143472
0.9885423944649201
0.9994663548520822
0.9970000904155318
0.9965715090968659
0.9961422379580201
0.9730295350302943
0.9979966371980692
0.9986809752691265
0.9087113161341686
0.9961554614076766
0.9937275591396599
0.9950097849512352
0.9578521180319195
0.9824494865380219
0.9283690092877047
0.9966006760418895
0.9992164773293071
0.8899609913439677
0.9234809554164983
0.9896968321133269
0.99930723195611

0.9941010586540802
0.9175114092848669
0.9989185793534685
0.9601130687830255
0.9823969046675953
0.987509353371624
0.9750284497589784
0.7670713559762049
0.9799040026417628
0.9964602382043547
0.9805162162706789
0.9956329099297624
0.8233656894079155
0.9981591063364792
0.9871884825698406
0.9816209481238578
0.9864141583558487
0.955979372506749
0.9859988800344116
0.13401774928409046
0.9670118763841975
0.9982360549225116
0.9405168162505936
0.8449630772019121
0.9794755673928973
0.9970493614065721
0.9968950051556178
0.982654119449989
0.9793043544976379
0.9889850716476672
0.9835255763956517
0.9909452747110228
0.9726749526137094
0.9902044169400354
0.9822967651719073
0.9397496266431167
0.9953781176104015
0.98528014196879
0.9734367308613266
0.9905773548256758
0.9646235470298518
0.9971084881522744
0.9923935855161122
0.12179167227945877
0.989153750862932
0.9945707486649417
0.9783773448864858
0.9861900023542787
0.9955981666192218
0.9976374886402096
0.955883063250773
0.9946648338071928
0.991837933931166

0.9686872309284182
0.8865556418429398
0.8421657174750894
0.9970551624769926
0.9792267153941402
0.9312107936653224
0.988709275220367
0.9963267467649028
0.9732449156443563
0.9864819928941603
0.9949181214461918
0.9965197486381431
0.9997733337096967
0.8756943919813384
0.9962719634903471
0.9238871427957651
0.9773767457037114
0.991140746696942
0.8940525325226162
0.9524510235966921
0.9923652245801021
0.9126131870792538
0.9070937382745753
0.9952065661250453
0.9695186382711987
0.9965114926029761
0.9784354149918646
0.9956690147474752
0.9122721297761082
0.9962198598709128
0.9911658624517655
0.8704317268791218
0.9784968695377609
0.990981650400374
0.8865787515205397
0.9998296959803278
0.9777199672321042
0.9722076073938327
0.8954239769382363
0.9910896045165802
0.8930668742308524
0.9965093356471786
0.9707433237211693
0.9854465179162929
0.9978877644562963
0.9602459175248316
0.9528152054039902
0.9937953236224109
0.9771113624995028
0.9841301988728897
0.9994048730758934
0.9964187398556577
0.9956430318002

0.9762901335854246
0.993589867082027
0.9574000276922102
0.9969315201705081
0.9991391317326004
0.9584254482676492
0.9951239576899128
0.9802289002909856
0.9978646563943655
0.902844338024158
0.9204393974175842
0.9945744072795282
0.9965749409426169
0.9375218140509752
0.9875216340342119
0.9991951348714067
0.9980250314253161
0.9658595945943336
0.9736647223655758
0.9918174628168372
0.9417407567787459
0.9657012221860442
0.9901613272146944
0.9990725767218077
0.9623033604198427
0.9968760753243208
0.9993409955385026
0.892975828412651
0.9997083658579045
0.9184412237364045
0.9943538326342044
0.9898749745492691
0.9988382685344023
0.9368694817743144
0.9873821176357023
0.9706635020373873
0.9947454652241793
0.9982184234180854
0.6395931095227583
0.9639812667537268
0.8986431779014199
0.9798461901971136
0.9628626851285655
0.9875487116088311
0.9947461095425882
0.9132807678997429
0.9789439750042535
0.9989594740376052
0.9228554429165257
0.9903048483638562
0.6737868773195533
0.9126625549786438
0.9348472873365

0.9986757929158966
0.853851700257652
0.9933491723744412
0.9829139561242378
0.9757022817551907
0.9991522076246412
0.9038041678289573
0.8806411583441285
0.9973150267298406
0.9616134475577885
0.9954387725977478
0.9989809002648457
0.9451672643946626
0.9731763112716442
0.995935513310286
0.9975015278677118
0.9882740100824497
0.9589621454708124
0.8583023366420102
0.9838073875209323
0.9984758837364832
0.9397340978627425
0.9991475732368363
0.9301903966937753
0.9967872370284937
0.9557986563083808
0.9798198309730096
0.9958829513867713
0.952778940767526
0.9592058350565105
0.9026990913820084
0.9982497216892229
0.9957908402069714
0.9465362633443595
0.9815527459970046
0.981041591196775
0.9991915903044234
0.9889350198847466
0.6912174018282885
0.9424782372558494
0.9891607280232153
0.9984287212993815
0.9764646827743871
0.9898812422708924
0.9982099113642267
0.9837217879124529
0.9926662702876371
0.9901415483684962
0.9865095781577046
0.8710593952428433
0.9628263936077361
0.9955358433074734
0.99741369254876

0.9681237069858017
0.9030208255173512
0.9949707165661585
0.9316894094374559
0.999925469024291
0.9590455442898838
0.9969858588589979
0.9990838721116875
0.7539766940487614
0.9231015280154663
0.9966122888297829
0.9889047588763001
0.9898226453238298
0.3709102484704525
0.9988979244179339
0.7270107433816547
0.19606458605886548
0.16400794122895607
0.07665897738860282
0.05990250776918898
0.996570019240485
0.992515140099873
0.9245875291576124
0.9877528230939646
0.9891253546709208
0.9949599831511902
0.9927902520366166
0.9750683777419984
0.9893930525009575
0.6643527596049753
0.9462455529816627
0.996131648813837
0.9387047744017013
0.9926591391925069
0.9969796736543713
0.9289258560582272
0.9962308430245552
0.9663702286454984
0.9882633118085904
0.9869549553495592
0.9614995935319258
0.15417469885992616
0.9813887837510186
0.9535742245070195
0.9950218261947368
0.998045272855485
0.9719286997702826
0.9796425211525311
0.947937776499004
0.9839167075669587
0.9539194247224015
0.9940495737245869
0.20279554391

0.9632608445295217
0.9958370242106362
0.4129675851106628
0.9999041384302134
0.6115725780751997
0.9502094765933783
0.9941902502049107
0.9799596689766649
0.959199332439704
0.9926051078997304
0.9979116971319565
0.9984384040074931
0.894048703345617
0.9014036282217083
0.9834221093148212
0.7935252362851982
0.992191784779155
0.9991939995470803
0.9868080656822062
0.9816331437318814
0.9587417406752266
0.04830716319115995
0.9277416618514382
0.8567135785591287
0.9879198511925565
0.9532210456159614
0.9807989545000022
0.9974854393798459
0.9991143555835538
0.990885578674119
0.5213021799922649
0.9866264029095814
0.9857578169497314
0.9992359997852605
0.750224277264848
0.9970312931834135
0.9969680735480051
0.9924929204977738
0.9918205558025384
0.9973981844213454
0.9915893004597929
0.952849860065699
0.9501034112527961
0.9997070208357282
0.9880485790411746
0.9897176572232548
0.98783043655464
0.9955015604465235
0.9960835699074824
0.9784309942076175
0.9951811862487883
0.9814291386026751
0.9972627402063157


In [27]:
for layer in neu_net.layers:
    print(layer.weights[:20])

[[3.55734918e+00 9.28508008e+00 3.44515219e+00 ... 3.09161599e+00
  4.69909087e+00 3.98153732e+00]
 [4.30485688e-03 4.95152477e-03 3.39412025e-03 ... 1.52013382e-03
  4.80949760e-04 1.79494154e-03]
 [4.90901218e-03 4.37049070e-03 1.60661318e-03 ... 1.11211221e-03
  4.91058373e-03 1.19108230e-03]
 ...
 [9.43322821e-04 5.73098747e-04 4.08307193e-03 ... 2.13394903e-05
  3.74566444e-04 1.15507490e-03]
 [3.60671260e-03 1.60020225e-04 2.87969353e-03 ... 9.34977332e-04
  4.70362406e-03 1.59368050e-03]
 [3.33360397e-03 1.13826416e-04 1.70335471e-03 ... 9.95354099e-04
  3.19237693e-04 3.69378950e-05]]
[[-0.69436837 -0.77363567 -0.67847928 -1.08429905 -0.98908355 -0.53217939
  -0.61965785 -0.43773163 -0.68727952 -0.38133811  0.83767158  0.52105247
  -0.46010076 -0.76541948 -0.51042473 -0.0750223  -0.63795517 -0.22345863
  -0.4396779   0.19270879 -0.49440452 -0.95596585 -0.81334987 -0.65134081
   0.24511194  1.82559744]
 [-1.56250936 -1.48850203 -2.25026066 -2.02511091 -2.17210551 -2.63637417
  -